In [1]:
#### Prepare Data With Only One Camera

In [2]:
'''
import csv
import cv2
import numpy as np
from PIL import Image

lines = []
with open('data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)

images = []
measurements = []
header = True

for line in lines:
    if header:
        header = False
        continue
    
    source_path = line[0]
    file_name = source_path.split('/')[-1]
    current_path = 'data/IMG/' + file_name
    #image = cv2.imread(current_path)
    image = np.asarray(Image.open(current_path))
    images.append(image)
    measurement = float(line[3])
    measurements.append(measurement)
    
    # augmenting data
    image_flipped = np.fliplr(image)
    images.append(image_flipped)
    measurement_flipped = -measurement
    measurements.append(measurement_flipped)
'''

"\nimport csv\nimport cv2\nimport numpy as np\nfrom PIL import Image\n\nlines = []\nwith open('data/driving_log.csv') as csvfile:\n    reader = csv.reader(csvfile)\n    for line in reader:\n        lines.append(line)\n\nimages = []\nmeasurements = []\nheader = True\n\nfor line in lines:\n    if header:\n        header = False\n        continue\n    \n    source_path = line[0]\n    file_name = source_path.split('/')[-1]\n    current_path = 'data/IMG/' + file_name\n    #image = cv2.imread(current_path)\n    image = np.asarray(Image.open(current_path))\n    images.append(image)\n    measurement = float(line[3])\n    measurements.append(measurement)\n    \n    # augmenting data\n    image_flipped = np.fliplr(image)\n    images.append(image_flipped)\n    measurement_flipped = -measurement\n    measurements.append(measurement_flipped)\n"

In [3]:
#### Prepare Data With All Cameras and Augmented Features

In [4]:
import csv
import cv2
import numpy as np
from PIL import Image

lines = []
car_images = []
steering_angles = []

with open('data/driving_log.csv') as f:
    reader = csv.reader(f)
    for row in reader:
        steering_center = float(row[3])

        # create adjusted steering measurements for the side camera images
        correction = 0.2 # this is a parameter to tune
        steering_left = steering_center + correction
        steering_right = steering_center - correction

        # read in images from center, left and right cameras
        path = "data/IMG/" # fill in the path to your training IMG directory
        img_center = np.asarray(Image.open(path + row[0].split('\\')[-1]))
        img_left = np.asarray(Image.open(path + row[1].split('\\')[-1]))
        img_right = np.asarray(Image.open(path + row[2].split('\\')[-1]))

        # add images and angles to data set
        car_images.extend([img_center, img_left, img_right])
        steering_angles.extend([steering_center, steering_left, steering_right])
        
        # augmenting data: flipping images and steering
        img_center_flipped = np.fliplr(img_center)
        img_left_flipped = np.fliplr(img_left)
        img_right_flipped = np.fliplr(img_right)
        car_images.extend([img_center_flipped, img_left_flipped, img_right_flipped])
        steering_center_flipped = -steering_center
        steering_left_flipped = -steering_left
        steering_right_flipped = -steering_right
        steering_angles.extend([steering_center_flipped, steering_left_flipped, steering_right_flipped])


In [5]:
X_train = np.array(car_images)

In [6]:
y_train = np.array(steering_angles)

In [7]:
#from keras.models import Sequential
#from keras.layers import Flatten, Dense, Lambda

#model = Sequential()
#model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))
#model.add(Flatten())
#model.add(Dense(1))

#model.compile(loss='mse', optimizer='adam')
#model.fit(X_train, y_train, validation_split=0.2, shuffle=True, epochs=2)

#model.save('model/first_20210301.h5')


In [8]:
import os
import csv
import cv2
import numpy as np

from PIL import Image
import keras
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda
from keras.layers.core import Dense, Activation, Flatten, Dropout

from keras.layers import Cropping2D
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras import backend as K

print(keras.__version__)

import socketio
print(socketio.__version__)

Using TensorFlow backend.


2.4.0
4.2.1


In [9]:
'''
The CNN architecture is used from NVIDIA's End to End Learning for Self-Driving Cars paper.
Reference: https://arxiv.org/pdf/1604.07316v1.pdf
'''
#Keras Sequential Model
model = Sequential()

#Image cropping to get rid of the irrelevant parts of the image (the hood and the sky)
model.add(Cropping2D(cropping=((50,20), (0,0)), input_shape=(160,320,3)))

#Pre-Processing the image
#model.add(Lambda(preprocess))
model.add(Lambda(lambda x: (x / 255.0) - 0.5))

#The layers
model.add(Conv2D(filters=24, kernel_size=(5, 5), strides=(2, 2),activation='relu'))
model.add(Conv2D(filters=36, kernel_size=(5, 5),strides=(2, 2), activation='relu'))
model.add(Conv2D(filters=48, kernel_size=(5, 5), strides=(2, 2),activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3, 3) ,activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3, 3),activation='relu'))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=50, activation='relu'))
model.add(Dense(units=10, activation='relu'))
model.add(Dense(units=1))
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cropping2d (Cropping2D)      (None, 90, 320, 3)        0         
_________________________________________________________________
lambda (Lambda)              (None, 90, 320, 3)        0         
_________________________________________________________________
conv2d (Conv2D)              (None, 43, 158, 24)       1824      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 77, 36)        21636     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 37, 48)         43248     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 6, 35, 64)         27712     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 4, 33, 64)         3

In [10]:
print(len(car_images))
print(len(steering_angles))

print(car_images[1].shape)
print(steering_angles[1])

32880
32880
(160, 320, 3)
0.2


In [11]:
model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, epochs=10)

model.save('model/first_20210317_10.h5')

print('The model.h5 file has been created!') 

Epoch 1/10
822/822 [==============================] - 418s 496ms/step - loss: 0.0649 - val_loss: 0.0314
Epoch 2/10
822/822 [==============================] - 245s 298ms/step - loss: 0.0490 - val_loss: 0.0290
Epoch 3/10
822/822 [==============================] - 245s 298ms/step - loss: 0.0403 - val_loss: 0.0339
Epoch 4/10
822/822 [==============================] - 244s 297ms/step - loss: 0.0356 - val_loss: 0.0350
Epoch 5/10
822/822 [==============================] - 248s 302ms/step - loss: 0.0309 - val_loss: 0.0333
Epoch 6/10
822/822 [==============================] - 249s 303ms/step - loss: 0.0290 - val_loss: 0.0321
Epoch 7/10
822/822 [==============================] - 247s 301ms/step - loss: 0.0257 - val_loss: 0.0327
Epoch 8/10
822/822 [==============================] - 248s 302ms/step - loss: 0.0244 - val_loss: 0.0329
Epoch 9/10
822/822 [==============================] - 251s 305ms/step - loss: 0.0225 - val_loss: 0.0325
Epoch 10/10
822/822 [==============================] - 247s 300m